### Import all libraries

In [9]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
pn.extension('tabulator') #for JS and CSS used wtih Tabulator widgets

In [10]:
df = pd.read_csv('bank_data.csv')

In [11]:
df

,id,first_name,last_name,email,gender,age,months_as_a_customer,join_date,leave_date,join_year,city,bank_balance,monthly_income,credit_score,is_active_customer
0,1,Zquqjsp,Kzinjdx,rexydrr@hotmail.com,Male,25,74,2016-08-07,2022-09-25,2016,Liverpool,31327,8000,407,0
1,2,Juwyddl,Dkdipwi,hkpywgq@outlook.com,Female,20,45,2019-07-06,NaN,2019,London,0,6000,508,1
2,3,Yjfjodr,Vbrtbkp,zojkhts@yahoo.com,Female,48,76,2015-03-15,2021-06-11,2015,Leeds,24657,9000,553,0
3,4,Sukpurx,Kyqvwtk,astmqkh@yahoo.com,Male,21,69,2017-07-13,NaN,2017,Leeds,0,6000,774,1
4,5,Zprbudk,Ogsmodp,bcayldc@gmail.com,Female,40,12,2021-11-26,2022-12-19,2021,Leeds,95824,6000,432,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Huvapwm,Hgwohjd,pbpnooo@gmail.com,Female,49,10,2020-12-27,2021-11-11,2020,Sheffield,99940,1000,386,0
9996,9997,Dsosftv,Mhkpqbk,dhjqpve@outlook.com,Female,37,94,2015-06-20,NaN,2015,Birmingham,0,8000,570,1
9997,9998,Majrzpn,Qvketpx,yemikfj@gmail.com,Female,21,18,2018-08-18,2020-03-05,2018,Edinburgh,12314,5000,556,0
9998,9999,Kvwfdnw,Agxazcj,kumgfjb@outlook.com,Male,51,46,2019-05-29,NaN,2019,Leeds,0,6000,801,1


# creating the dashboard elements / widgets

### 1. A slider to select year

Get all the joining dates and leaving dates of customers. From those dates, we pick the max and min year values.

#### get max and min of join_date

In [12]:
join_dates = df['join_date']

In [13]:
years_join = join_dates.str[:4]
years_join = years_join.astype(int)
min_year_join = years_join.min()
max_year_join = years_join.max()

In [14]:
min_year_join = int(min_year_join)
min_year_join

2015

In [15]:
max_year_join = int(max_year_join)
max_year_join

2021

#### get max and min of leave_date

In [16]:
leave_dates = df['leave_date']

In [17]:
years_left = leave_dates.str[:4]
years_left = years_left.dropna().astype(int)
min_year_left = years_left.min()
max_year_left = years_left.max()

In [18]:
min_year_left = int(min_year_left)
min_year_left

2015

In [19]:
max_year_left = int(max_year_left)
max_year_left

2023

### the year sliders

1. join date slider

In [109]:
join_slider = pn.widgets.IntSlider(name='Join Date', start=min_year_join,end=max_year_join,value=min_year_join)
join_slider

IntSlider(end=2021, name='Join Date', start=2015, value=2015)

2. leave date slider

In [21]:
leave_slider = pn.widgets.IntSlider(name='Join Date', start=min_year_left,end=max_year_left,value=min_year_left)
leave_slider

IntSlider(end=2023, name='Join Date', start=2015, value=2015)

### create two separte dataframes for city based customer analysis
active and inactive

In [22]:
active_cust_df = df[df['is_active_customer'] == 1].groupby('city').agg({'monthly_income': 'mean', 'credit_score': 'mean'}).reset_index()
inactive_cust_df = df[df['is_active_customer'] == 0].groupby('city').agg({'monthly_income': 'mean', 'credit_score': 'mean'}).reset_index()

In [23]:
#round the values to 2 decimal places
active_cust_df = active_cust_df.round({'monthly_income': 2, 'credit_score': 2})
inactive_cust_df = inactive_cust_df.round({'monthly_income': 2, 'credit_score': 2})

In [24]:
active_cust_df

,city,monthly_income,credit_score
0,Birmingham,5543.86,575.65
1,Bradford,5640.00,591.40
2,Bristol,5562.72,579.76
3,Edinburgh,5430.71,563.06
4,Glasgow,5319.70,575.93
5,Leeds,5433.33,576.80
6,Liverpool,5576.09,571.39
7,London,5369.23,582.24
8,Manchester,5656.86,583.88
9,Other,5661.92,581.57


In [25]:
inactive_cust_df

,city,monthly_income,credit_score
0,Birmingham,5568.39,570.12
1,Bradford,5490.88,574.49
2,Bristol,5502.48,574.53
3,Edinburgh,5496.64,571.59
4,Glasgow,5385.17,581.97
5,Leeds,5454.40,574.24
6,Liverpool,5447.83,572.81
7,London,5553.09,575.68
8,Manchester,5484.85,568.62
9,Other,5455.27,576.63


#### rename the columns

In [26]:
active_cust_df['active_customers_avg_monthly_income'] = active_cust_df['monthly_income']
active_cust_df['active_customers_avg_credit_score'] = active_cust_df['credit_score']

In [27]:
inactive_cust_df['inactive_customers_avg_monthly_income'] = inactive_cust_df['monthly_income']
inactive_cust_df['inactive_customers_avg_credit_score'] = inactive_cust_df['credit_score']

#### drop the old columns

In [28]:
active_cust_df = active_cust_df.drop('monthly_income', axis=1)
active_cust_df = active_cust_df.drop('credit_score', axis=1)
inactive_cust_df = inactive_cust_df.drop('monthly_income', axis=1)
inactive_cust_df = inactive_cust_df.drop('credit_score', axis=1)

In [29]:
active_cust_df

,city,active_customers_avg_monthly_income,active_customers_avg_credit_score
0,Birmingham,5543.86,575.65
1,Bradford,5640.00,591.40
2,Bristol,5562.72,579.76
3,Edinburgh,5430.71,563.06
4,Glasgow,5319.70,575.93
5,Leeds,5433.33,576.80
6,Liverpool,5576.09,571.39
7,London,5369.23,582.24
8,Manchester,5656.86,583.88
9,Other,5661.92,581.57


In [30]:
inactive_cust_df

,city,inactive_customers_avg_monthly_income,inactive_customers_avg_credit_score
0,Birmingham,5568.39,570.12
1,Bradford,5490.88,574.49
2,Bristol,5502.48,574.53
3,Edinburgh,5496.64,571.59
4,Glasgow,5385.17,581.97
5,Leeds,5454.40,574.24
6,Liverpool,5447.83,572.81
7,London,5553.09,575.68
8,Manchester,5484.85,568.62
9,Other,5455.27,576.63


# graphs

In [119]:
yaxis_credit_score = pn.widgets.RadioButtonGroup(name='y-axis', options=['credit_score','monthly_income'],button_type='success')

Connect data pipeline with widets

##### Convert df to interactive df. We do this to make it possible to use the our data in for interactive visualizations.

In [86]:
idf = df.interactive()

In [41]:
idf['is_active_customer']

In [87]:
join_date_years = idf.join_date.str[:4]

In [88]:
type(join_slider)

panel.widgets.slider.IntSlider

In [89]:
type(join_date_years)

hvplot.interactive.Interactive

In [90]:
cities = ['London', 'Birmingham', 'Leeds', 'Glasgow', 'Sheffield',
          'Bradford', 'Manchester', 'Edinburgh', 'Liverpool', 'Bristol', 'Other']
#top 10 cities by population
#src: https://worldpopulationreview.com/countries/cities/united-kingdom

In [91]:
idf.columns

### Checking credit scores and monthly incomes by cities

In [126]:
data_pipeline = (
    idf[
        (idf.join_year <= join_slider) &
        (idf.city.isin(cities))
    ]
    .groupby(['city','join_year'])[yaxis_credit_score].mean()
    .to_frame()
    .reset_index()
    .sort_values(by=['city', 'join_year'])
    .reset_index(drop=True)
)

In [127]:
data_pipeline

### Create an hvplot to visualize the above data

In [128]:
credit_scores_and_incomes = data_pipeline.hvplot(x = 'join_year', by='city', y=yaxis_credit_score,line_width=2, title="Credit Scores and Incomes by City")
credit_scores_and_incomes

In [129]:
type(data_pipeline)

hvplot.interactive.Interactive

In [125]:
score_and_income_by_city = data_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
score_and_income_by_city

## Credit Score vs Income

In [157]:
score_vs_income_pipeline = (
    idf[
        (idf.join_year == join_slider) &
        ((idf.city.isin(cities)))
    ]
    .groupby(['city', 'join_year', 'monthly_income'])['credit_score'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='city')  
    .reset_index(drop=True)
)
score_vs_income_pipeline

### Visulize the data above with a scatter plot

In [158]:
score_vs_income_scatterplot = score_vs_income_pipeline.hvplot(x='monthly_income', 
                                                                y='credit_score', 
                                                                by='city', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
score_vs_income_scatterplot

## Bar Plot

In [138]:
df

,id,first_name,last_name,email,gender,age,months_as_a_customer,join_date,leave_date,join_year,city,bank_balance,monthly_income,credit_score,is_active_customer
0,1,Zquqjsp,Kzinjdx,rexydrr@hotmail.com,Male,25,74,2016-08-07,2022-09-25,2016,Liverpool,31327,8000,407,0
1,2,Juwyddl,Dkdipwi,hkpywgq@outlook.com,Female,20,45,2019-07-06,NaN,2019,London,0,6000,508,1
2,3,Yjfjodr,Vbrtbkp,zojkhts@yahoo.com,Female,48,76,2015-03-15,2021-06-11,2015,Leeds,24657,9000,553,0
3,4,Sukpurx,Kyqvwtk,astmqkh@yahoo.com,Male,21,69,2017-07-13,NaN,2017,Leeds,0,6000,774,1
4,5,Zprbudk,Ogsmodp,bcayldc@gmail.com,Female,40,12,2021-11-26,2022-12-19,2021,Leeds,95824,6000,432,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Huvapwm,Hgwohjd,pbpnooo@gmail.com,Female,49,10,2020-12-27,2021-11-11,2020,Sheffield,99940,1000,386,0
9996,9997,Dsosftv,Mhkpqbk,dhjqpve@outlook.com,Female,37,94,2015-06-20,NaN,2015,Birmingham,0,8000,570,1
9997,9998,Majrzpn,Qvketpx,yemikfj@gmail.com,Female,21,18,2018-08-18,2020-03-05,2018,Edinburgh,12314,5000,556,0
9998,9999,Kvwfdnw,Agxazcj,kumgfjb@outlook.com,Male,51,46,2019-05-29,NaN,2019,Leeds,0,6000,801,1


In [167]:
scores_and_incomes = pn.widgets.RadioButtonGroup(
    name='y-axis',
    options=['monthly_income','credit_score'],
    button_type='success'
)

In [166]:
score_vs_income_barplot = score_vs_income_pipeline.hvplot(x='monthly_income', 
                                                                y='credit_score', 
                                                                by='city', 
                                                                size=80, kind="bar", 
                                                                alpha=0.7,
                                                                legend=True, 
                                                                height=500, 
                                                                width=500)
score_vs_income_barplot

# Creating the Dashboard with All the Widgets Created

In [212]:
main_dashboard = pn.template.FastGridTemplate(
    title = 'Bank Customers Dashboard',
    sidebar = [pn.pane.Markdown("Incomes and Credit Scores"),
               pn.pane.PNG('https://icons.iconarchive.com/icons/graphicloads/100-flat/256/bank-icon.png'),
               pn.pane.Markdown("## Year"),
               join_slider],
    main=[pn.Row(pn.Column(yaxis_credit_score, 
                           credit_scores_and_incomes.panel(width=700), margin=(0,25)), 
                 score_and_income_by_city.panel(width=500)),
          pn.Row(pn.Column(score_vs_income_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(scores_and_incomes, score_vs_income_barplot.panel(width=600)))],
    accent_base_color="#43C09A",
    header_background="#43C09A"
)
main_dashboard.show()

Launching server at http://localhost:60721
